In [ ]:
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='', #sua senha
    database='musica', #Nome database que quer acessar
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

In [ ]:
#Visualizando as tabelas
db('SHOW TABLES')

In [ ]:
#Visualizando o schema
db('DESCRIBE musica')

## Pipeline do comando Select ##
```
SELECT [DISTINCT] coluna(s)
FROM tabela(s)
WHERE condição
GROUP BY grupo de uma coluna
HAVING condição (aplicada após GROUP BY)
ORDER BY expressao (pode ser coluna, ASC OU DESC)
LIMIT nº max resultados
OFFSET onde quer começar o limit
```  


A ordem de execução do comando `SELECT` é aproximadamente como segue:

1. `FROM <source_tables>`: indica as tabelas que serão usadas nesta query e, conceitualmente, combina estas tabelas através de *produto cartesiano* em uma grande tabela. (Note o termo "*conceitualmente*" que usei: em termos de implementação da query este produto cartesiano raramente é construído.)

2. `WHERE <filter_expression>`: filtra linhas.

3. `GROUP BY <grouping_expressions>`: agrupa conjuntos de linhas.

4. `SELECT <select_heading>`: escolha de colunas e de agregados.

5. `HAVING <filter_expression>`: outra filtragem, esta aplicada apenas **depois** da agregação. Pode usar resultados do processo de agregação. Obriga o uso de `GROUP BY`.

6. `DISTINCT`: Elimina linhas duplicadas.

7. `ORDER BY`: ordena as linhas do resultado.

8. `OFFSET <count>`: Pula linhas do resultado. Requer LIMIT.

9. `LIMIT <count>`: Mantém apenas um número máximo de linhas.

Esta sequencia também serve como dica de como projetar uma query! 
- Comece identificando as tabelas que você deseja usar
- Monte o filtro de linhas, incluindo critérios de `JOIN`
- Agrupe
- Selecione colunas e aplique funções de agregação, conforme necessário
- Filtre com `HAVING`, agora que temos agregação
- O resto é mais fácil, aplique conforme requerido

## Comando Create

In [ ]:
db('DROP TABLE IF EXISTS Usuario')
db('''CREATE TABLE Usuario(
    nome VARCHAR(45),
    usuario_id INT NOT NULL AUTO_INCREMENT,
    musica_favorita_id INT,
    PRIMARY KEY (usuario_id),
    CONSTRAINT fk_musica_favorita FOREIGN KEY (musica_favorita_id)
        REFERENCES musica (Codigo_musica)
  )''')

## Comando Insert

In [ ]:
db('''INSERT INTO Usuario (nome, musica_favorita_id) VALUES 
   ("Joao", 9 ),
   ("Carla", 1),
   ("Tiago", 19),
   ("Tomas", 19),
   ("Tatiana", 19),
   ("Temer", 19)
   ''')

In [ ]:
db('SELECT * FROM Usuario')

## Comando Update

In [ ]:
db('UPDATE Usuario SET musica_favorita_id = 5 WHERE nome LIKE "T%"')

In [ ]:
db('SELECT * FROM Usuario')

## Joins + Temporary Table
### Inner Join
#### Juntar tabelas se e somente se  os valores que quer comparar existirem em ambas
#### Exemplo: Tabela com nome da pessoa e nome da música favorita
#### OBS: dropar a tabela quando terminar de usar

In [ ]:
db('''DROP TABLE IF EXISTS Usuario_FavSong''')
db('''CREATE TEMPORARY TABLE Usuario_FavSong SELECT nome, Nome_musica FROM Usuario 
    INNER JOIN musica ON Usuario.musica_favorita_id = musica.Codigo_Musica
    ''')

In [ ]:
db("SELECT * FROM Usuario_FavSong")

### Outer Left e Right Joins

In [ ]:
db('''DROP TABLE IF EXISTS musicas_nofav ''')
db('''CREATE TEMPORARY TABLE  musicas_nofav SELECT Nome_Musica FROM musica LEFT OUTER JOIN Usuario 
    ON Usuario.musica_favorita_id = musica.Codigo_Musica
    WHERE Usuario.usuario_id IS NULL
    ''')

In [ ]:
db("SELECT * FROM musicas_nofav LIMIT 2")

### GROUP BY

In [ ]:
### Selecionando duração total de trilhas por autor
db('''SELECT sum(Duracao), nome_autor FROM musica 
INNER JOIN musica_autor USING(codigo_musica) 
INNER JOIN autor USING(Codigo_autor) 
GROUP BY nome_autor LIMIT 5''')

In [ ]:
### Agrupa os nomes de usuários se tiverem as mesmas músicas favoritas
db('''SELECT GROUP_CONCAT(nome SEPARATOR ", ") FROM Usuario GROUP BY musica_favorita_id 
''')

In [ ]:
db('''DROP TABLE IF EXISTS teste ''')

db('''CREATE TEMPORARY TABLE  teste SELECT 
    Codigo_autor, Nome_autor, COUNT( DISTINCT Codigo_cd) as cnt
    FROM autor
    INNER JOIN musica_autor USING(Codigo_autor)
    INNER JOIN musica USING(Codigo_Musica)
    INNER JOIN faixa USING(Codigo_Musica)
    INNER JOIN cd USING(Codigo_cd)
    GROUP BY Codigo_autor
    ORDER BY cnt
    DESC
''')

In [ ]:
db('''describe teste ''')

In [ ]:
db("SELECT * FROM TESTE")

## Modelo Relacional

![](ModeloRelacional.png)

## Operador LIKE

![](LikeOperator.png)

In [ ]:
db("SELECT * FROM CD")

In [ ]:
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='Nalacristina2003', #sua senha
    database='pandemia', #Nome database que quer acessar
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

In [ ]:
db("SHOW TABLES FROM pandemia")

In [ ]:
db("SELECT * FROM vacina")

In [ ]:
db('''SELECT DISTINCT marca FROM vacina 
    INNER JOIN dose USING(id_vacina) 
    INNER JOIN pessoa USING(id_pessoa)
    INNER JOIN pessoasintoma USING(id_pessoa)
    INNER JOIN sintoma USING(id_sintoma)
    WHERE (pessoasintoma.data_ocorrencia - dose.data_aplicacao) <= 5
    ''')

In [ ]:
db('''SELECT id_pessoa, nome, vacina.doses, COUNT(id_pessoa)  as cnt
    FROM pessoa 
    LEFT OUTER JOIN dose USING(id_pessoa)
    INNER JOIN vacina USING(id_vacina)
    GROUP BY id_pessoa
    HAVING(cnt < vacina.doses)
    ''')

db('''
DROP VIEW IF EXISTS marca_pessoas;
''')
db('''
CREATE VIEW marca_pessoas AS
    SELECT marca , COUNT(DISTINCT id_pessoa) FROM vacina
    INNER JOIN dose USING(id_vacina)
    INNER JOIN pessoa USING(id_pessoa)
    GROUP BY id_vacina
    ''')

In [ ]:
db("SELECT * FROM marca_pessoas")

In [ ]:
connection.close()

## Dependências funcionais

### 𝑋 → 𝑌 ⇔ duas linhas da tabela que concordam em 𝑋 deverão concordar em 𝑌


## Chaves

### Superchave: um conjunto de atributos 𝐾 é uma superchave da relação se 𝐾 + é o conjunto completo de atributos desta relação

### Chave candidata: uma superchave irredutível, ou seja, nenhum atributo pode ser removido da superchave sem que seu status de superchave seja cancelado

### Chave primária: a chave candidata escolhida como identificador da relação

## Formas normais

### Ver slides aula 14/15/16


## Modelo entidade-relacionamento

### Ver slides aula 16

## Transaction

In [ ]:
##Transaction garante que ou todas as querys de um bloco sejam realziadas ou que nenhuma delas seja
##Exemplo:

try:
    db('START TRANSACTION')
    db('INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 2)')
    db('INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 1000)')
    db('COMMIT')
except Exception as e:
    print(e)
    db('ROLLBACK')

## ACID

### Atomicidade: garantir que um conjunto de comandos é executado ou rejeitado como uma única unidade.

### Consistência: indica que o banco de dados passa de um estado válido para outro estado válido a cada transação.

### Isolamento: capacidade de executar várias transações concorrentes sem que uma transação interfira diretamente na outra, ou seja, o usuário tem a percepção de que as transações foram executadas sequencialmente (ler slides aula 19).

### Durabilidade: quando uma transação é confirmada (COMMIT), ela permanecerá gravada mesmo que a energia acabe ou o sistema trave.

## Stored procedures, triggers, views, functions
#### (Ver notebook aula 20)

##  Programação funcional

### Programação funcional descreve o que queremos  calcular sem especificar como iterar sobre  os dados ( facilmente paralelizável).

### Programação funcional usa funções puras -> Robustez: se um bloco de cálculo falha (a máquina cai), podemos reiniciar o cálculo daquele bloco apenas, sem problemas.

### Ausência de estado global -> Facilita uso de memória distribuída.